In [ ]:
#Note: will need to make adjustments if averaging across seeds or using 1 seed. Starting with one seed.

In [ ]:
import sys
sys.path.append("../")
sys.path.append("../Modules/")

import os

import numpy as np
import analysis
import pyspike as spk

%cd ../scripts/

In [ ]:
base_directory = '/path/to/simulations/'  # Replace with the actual path to your simulation base directory
standard_directory = os.path.join(base_directory, 'Complex') # the directory of the main simulation without reduction for calculating errors

Compute a stat table. 
First row: simulation name
2. Somatic spike error (1-spike synchrony)
3. (sorted table by this ascending): dSpike error (sta R2) ( 1 for each dSpike relationship)
4. Runtime ratio = runtime/sim_duration
5. number of segments
6. nseg_by_lambda (NaN if not optimized by lambda)
7-9. Regional Degree of reductions (basal, oblique, tuft)
10. Mapping
11. LFP Error
12. Passive property error
13. Build time ratio (buildtime/sim_duration)
14. np seed
15. neuron seed

In [ ]:
# helper functions
# simulation_folders = [folder for folder in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, folder))]

def get_all_directories_within(directory):
    try:
        # List all items in the given directory
        items = os.listdir(directory)
        # Filter out only directories and include the original directory in the path
        directories = [os.path.join(directory, item) for item in items if os.path.isdir(os.path.join(base_directory, item))]
        return directories
    except Exception as e:
        raise(e)
# consider if these are needed    
# grouped_directories = group_directories_by_cell_and_seed(sim_directories)
# sta_data, r_squareds, maes, mses = analyze_simulation_directories(grouped_directories)

In [ ]:
import os
import pandas as pd
# functions for stats

# Placeholder function to calculate individual statistics
def calculate_somatic_spike_error(standard_simulation: str, simulation_folder: str):
    parameters = analysis.DataReader.load_parameters(simulation_folder)
    spike_trains = [spk.SpikeTrain(spike_times=analysis.DataReader.read_data(standard_simulation, "soma_spikes"), edges = parameters.h_tstop), 
                spk.SpikeTrain(spike_times=analysis.DataReader.read_data(simulation_folder, "soma_spikes"), edges = parameters.h_tstop)]
    f = spk.spike_sync_profile(spike_trains[0], spike_trains[1])
    return f.avrg()

def calculate_dSpike_error(simulation_folder):
    # Replace this with actual logic to compute dSpike error (sta R2)
    return 0.12  # Example placeholder value

def calculate_runtime_ratio(simulation_folder, parameters): 
    # compute runtime ratio (runtime/sim_duration)
    sim_duration = parameters.tstop
    # get simulation runtime; read parameters.path/simulation_runtime.txt
    simulation_runtime_path = f"{simulation_folder}/simulation_runtime.txt" # @MARK make sure that this is total runtime.
    with open(simulation_runtime_path, 'r') as file:
        sim_runtime = file.read()
    runtime_ratio = sim_runtime/sim_duration
    return runtime_ratio

def calculate_number_of_segments(simulation_folder):
    # Replace this with actual logic to compute number of segments
    return 100  # Example placeholder value

def calculate_nseg_by_lambda(parameters):
    # calculate nseg_by_lambda, NaN if not optimized by lambda
    if parameters.optimize_nseg_by_lambda:
        return parameters.segs_per_lambda
    else:
        return float('nan')

def calculate_regional_reduction(region, parameters):
    # Replace this with actual logic to compute regional degree of reductions (basal, oblique, tuft)
    if region == 'basal':
        return parameters.reduce_basal
    elif region == 'oblique':
        return parameters.reduce_oblique
    elif region == 'tuft':
        return parameters.reduce_tufts
    elif region == 'apic':
        return parameters.reduce_apic

def calculate_mapping(parameters):
    # compute mapping
    return parameters.synapse_mapping

def calculate_LFP_error(simulation_folder):
    # Replace this with actual logic to compute LFP error
    return 0.05  # Example placeholder value

def calculate_passive_property_error(simulation_folder):
    # Replace this with actual logic to compute passive property error
    return 0.1  # Example placeholder value

def calculate_build_time_ratio(simulation_folder, parameters):
    # compute build time ratio (buildtime/sim_duration)
    sim_duration = parameters.tstop
    # get simulation runtime; read parameters.path/simulation_runtime.txt
    builder_runtime_path = f"{simulation_folder}/builder_runtime.txt" # @MARK make sure that this is total runtime.
    with open(builder_runtime_path, 'r') as file:
        builder_runtime = file.read()
    runtime_ratio = builder_runtime/sim_duration
    return runtime_ratio

# Function to create and fill the statistics table
def create_statistics_for_simulations(base_directory):
    """
    Create a table of statistics for each simulation in the base directory.
    
    Parameters:
    base_directory (str): Path to the base directory containing simulation subfolders.
    
    Returns:
    pd.DataFrame: A DataFrame with calculated statistics for each simulation.
    """
    # Get list of simulation subfolders
    simulation_folders = get_all_directories_within(base_directory)

    # Initialize a DataFrame to hold the statistics
    columns = ['Simulation Name', 'Somatic Spike Error', 'dSpike Error', 'Runtime Ratio', 'Number of Segments',
               'nseg_by_lambda', 'Basal Reduction', 'Oblique Reduction', 'Tuft Reduction', 'Mapping', 
               'LFP Error', 'Passive Property Error', 'Build Time Ratio']
    statistics_table = pd.DataFrame(columns=columns)

    # Loop over each simulation folder to calculate and add statistics
    for simulation in simulation_folders:
        simulation_path = os.path.join(base_directory, simulation)
        parameters = analysis.DataReader.load_parameters(simulation_path)
        
        # Calculate statistics
        somatic_spike_error = calculate_somatic_spike_error(simulation_path, standard_directory)
        dSpike_error = calculate_dSpike_error(simulation_path, standard_directory)
        runtime_ratio = calculate_runtime_ratio(simulation_path, parameters)
        number_of_segments = calculate_number_of_segments(simulation_path)
        nseg_by_lambda = calculate_nseg_by_lambda(parameters)
        basal_reduction = calculate_regional_reduction('basal', parameters)
        apic_reduction = calculate_regional_reduction('apic', parameters)
        oblique_reduction = calculate_regional_reduction('oblique', parameters)
        tuft_reduction = calculate_regional_reduction('tuft', parameters)
        mapping = calculate_mapping(simulation_path)
        LFP_error = calculate_LFP_error(simulation_path, standard_directory)
        # passive_property_error = calculate_passive_property_error(simulation_path, standard_directory)
        build_time_ratio = calculate_build_time_ratio(simulation_path)

        # Add statistics to the table
        statistics_table = statistics_table.append({
            'Simulation Name': simulation,
            'Somatic Spike Error': somatic_spike_error,
            'dSpike Error': dSpike_error,
            'Runtime Ratio': runtime_ratio,
            'Number of Segments': number_of_segments,
            'nseg_by_lambda': nseg_by_lambda,
            'Basal Reduction': basal_reduction,
            'Apic Reduction': apic_reduction,
            'Oblique Reduction': oblique_reduction,
            'Tuft Reduction': tuft_reduction,
            'Mapping': mapping,
            'LFP Error': LFP_error,
            # 'Passive Property Error': passive_property_error,
            'Build Time Ratio': build_time_ratio
        }, ignore_index=True)

    # Sort the table by dSpike Error (ascending)
    statistics_table = statistics_table.sort_values(by='dSpike Error', ascending=True)

    return statistics_table

# Generate the statistics table for all simulations
statistics_table = create_statistics_for_simulations(base_directory)

# Display the final statistics table
print(statistics_table)
